Meta Learning

NumerAi

Tanmay Singh
2021569
CSAI
Class of '25

In [4]:
import os
import gc
import time
import json
import pickle
import numpy as np
import pandas as pd
import seaborn as sb
import xgboost as xgb
import lightgbm as lgb
import cloudpickle as cp
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset

In [ ]:
from tqdm.auto import tqdm
from scipy import stats
from numerapi import NumerAPI
from scipy.stats import pearsonr
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [7]:
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.decomposition import *
from sklearn.preprocessing import *
from sklearn.neural_network import *
from sklearn.model_selection import *
from sklearn.cluster._kmeans import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

Creating a Feature Set

In [11]:
feature_metadata = json.load(open(f"./data/v5.0/features.json"))

for metadata in feature_metadata:
  print(metadata, len(feature_metadata[metadata]))

2024-11-04 08:30:05,021 INFO numerapi.utils: target file already exists
2024-11-04 08:30:05,022 INFO numerapi.utils: download complete


feature_sets 17
targets 37


In [12]:
feature_sets = feature_metadata["feature_sets"]

for feature_set in ["small", "medium", "all"]:
  print(feature_set, len(feature_sets[feature_set]))

small 42
medium 705
all 2376


In [13]:
feature_sets = feature_metadata["feature_sets"]
feature_sets.keys()

dict_keys(['small', 'medium', 'all', 'v2_equivalent_features', 'v3_equivalent_features', 'fncv3_features', 'intelligence', 'charisma', 'strength', 'dexterity', 'constitution', 'wisdom', 'agility', 'serenity', 'sunshine', 'rain', 'midnight'])

In [14]:
for feature_set in feature_sets:
  print(f'Feature Set: {feature_set:<25}', f'Size: {len(feature_sets[feature_set])}')

Feature Set: small                     Size: 42
Feature Set: medium                    Size: 705
Feature Set: all                       Size: 2376
Feature Set: v2_equivalent_features    Size: 304
Feature Set: v3_equivalent_features    Size: 1000
Feature Set: fncv3_features            Size: 400
Feature Set: intelligence              Size: 35
Feature Set: charisma                  Size: 290
Feature Set: strength                  Size: 135
Feature Set: dexterity                 Size: 51
Feature Set: constitution              Size: 335
Feature Set: wisdom                    Size: 140
Feature Set: agility                   Size: 145
Feature Set: serenity                  Size: 95
Feature Set: sunshine                  Size: 325
Feature Set: rain                      Size: 666
Feature Set: midnight                  Size: 244


Loading the Training Set, with a 'medium' feature set

In [15]:
feature_set = feature_sets["medium"]

train = pd.read_parquet(
    f"./data/v5.0/train.parquet",
    columns=["era", "target"] + feature_set
)

2024-11-04 08:30:05,600 INFO numerapi.utils: target file already exists
2024-11-04 08:30:05,601 INFO numerapi.utils: download complete


Preprocessing the Training Set

In [16]:
train.rename(columns=lambda x: f'feature {feature_set.index(x)}' if x in feature_set else x, inplace=True)
feature_set = train.columns.drop(["era", "target"])

In [17]:
train['era'] = train['era'].astype('int32')

In [18]:
train

,era,target,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n0007b5abb0c3a25,1,0.25,1,2,3,2,3,2,2,2,...,2,0,3,2,4,3,2,1,0,0
n003bba8a98662e4,1,0.25,3,2,4,1,0,2,3,2,...,2,0,0,2,0,0,2,0,0,0
n003bee128c2fcfc,1,0.75,1,2,0,2,4,2,0,2,...,2,3,3,2,2,3,2,2,2,4
n0048ac83aff7194,1,0.25,1,2,3,4,0,2,3,2,...,2,0,2,1,1,4,2,0,2,1
n0055a2401ba6480,1,0.25,3,2,3,4,1,2,4,2,...,2,1,3,3,2,4,2,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffcc1dbdf2212e6,574,0.75,4,4,3,0,2,0,0,2,...,4,0,4,3,1,2,1,4,4,2
nffde3b371d67394,574,0.25,4,4,3,0,2,4,1,2,...,3,0,4,0,2,2,2,1,3,3
nfff1a1111b35e84,574,0.50,3,0,1,2,3,2,1,0,...,4,2,4,1,3,4,0,4,0,4


In [ ]:
train.isna().any().any()

In [ ]:
train = train.dropna(subset=['target'])
train

In [19]:
unique_era = train['era'].unique()

In [20]:
train[train['era'] == unique_era[0]]

,era,target,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n0007b5abb0c3a25,1,0.25,1,2,3,2,3,2,2,2,...,2,0,3,2,4,3,2,1,0,0
n003bba8a98662e4,1,0.25,3,2,4,1,0,2,3,2,...,2,0,0,2,0,0,2,0,0,0
n003bee128c2fcfc,1,0.75,1,2,0,2,4,2,0,2,...,2,3,3,2,2,3,2,2,2,4
n0048ac83aff7194,1,0.25,1,2,3,4,0,2,3,2,...,2,0,2,1,1,4,2,0,2,1
n0055a2401ba6480,1,0.25,3,2,3,4,1,2,4,2,...,2,1,3,3,2,4,2,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nfe717abdac4977b,1,0.50,2,2,3,4,1,2,3,2,...,2,4,0,0,1,4,2,0,0,0
nfe91b60b73a2803,1,0.50,2,2,4,0,0,2,1,2,...,2,0,1,4,4,4,2,1,2,0
nfea11c308589112,1,0.75,0,2,4,0,0,2,4,2,...,2,4,0,1,0,4,2,0,0,0


In [21]:
dataset = train
dataset

,era,target,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n0007b5abb0c3a25,1,0.25,1,2,3,2,3,2,2,2,...,2,0,3,2,4,3,2,1,0,0
n003bba8a98662e4,1,0.25,3,2,4,1,0,2,3,2,...,2,0,0,2,0,0,2,0,0,0
n003bee128c2fcfc,1,0.75,1,2,0,2,4,2,0,2,...,2,3,3,2,2,3,2,2,2,4
n0048ac83aff7194,1,0.25,1,2,3,4,0,2,3,2,...,2,0,2,1,1,4,2,0,2,1
n0055a2401ba6480,1,0.25,3,2,3,4,1,2,4,2,...,2,1,3,3,2,4,2,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffcc1dbdf2212e6,574,0.75,4,4,3,0,2,0,0,2,...,4,0,4,3,1,2,1,4,4,2
nffde3b371d67394,574,0.25,4,4,3,0,2,4,1,2,...,3,0,4,0,2,2,2,1,3,3
nfff1a1111b35e84,574,0.50,3,0,1,2,3,2,1,0,...,4,2,4,1,3,4,0,4,0,4


In [22]:
dataset.isna().any().any()

False

In [23]:
dataset['target'].value_counts()

target
0.50    1373108
0.25     550995
0.75     550373
1.00     136126
0.00     135668
Name: count, dtype: int64

Splitting the Training Set into Training & Validation Fractions

In [24]:
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

print(f'Train Split: {train_df.shape}')
print(f'Val Split: {val_df.shape}')

Train Split: (2197016, 707)
Val Split: (549254, 707)


Clearing the unused variables using Python's Garbage Collector to free up RAM

In [25]:
del train
gc.collect()

0

In [26]:
train_df['target'].value_counts()

target
0.50    1098941
0.25     440350
0.75     440295
1.00     108869
0.00     108561
Name: count, dtype: int64

In [27]:
val_df['target'].value_counts()

target
0.50    274167
0.25    110645
0.75    110078
1.00     27257
0.00     27107
Name: count, dtype: int64

In [28]:
train_df

,era,target,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n02ccde8415c38aa,538,0.50,1,2,0,2,1,2,0,3,...,4,4,1,4,0,1,0,4,3,4
n2304b3e75684be7,538,0.75,4,3,4,0,0,4,4,0,...,2,2,4,3,0,3,1,0,2,0
n8f9f03f87006ca6,301,0.25,3,0,0,0,2,1,4,4,...,0,2,3,1,4,1,0,1,1,1
n329f28c70b122ca,516,0.50,1,4,1,1,0,2,3,0,...,4,1,4,3,4,1,2,2,4,0
neb7b6c30352e104,336,0.50,2,1,0,2,4,3,4,4,...,2,4,3,4,4,3,2,4,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
n26ad4563c77aadd,40,0.50,3,2,1,2,3,2,3,2,...,2,2,4,4,1,4,2,1,2,3
nd17176101e0c0d4,369,0.50,3,0,1,3,3,2,1,2,...,2,3,3,3,3,1,0,2,0,4
n68766810d412555,500,0.50,4,3,3,2,2,4,2,4,...,0,2,3,0,0,2,1,2,2,1


In [29]:
val_df

,era,target,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
nf313a6a2ba5d8ea,308,0.50,1,0,4,0,0,2,3,4,...,1,0,0,4,0,0,0,2,0,0
ne898d680d03f464,331,0.50,3,0,1,1,2,0,3,1,...,1,4,2,0,2,2,1,1,2,3
n986fb66ab539cb7,237,0.50,1,3,0,2,3,1,3,2,...,1,3,2,4,2,2,0,4,2,3
n3d14ff5f4be25e3,125,0.50,3,3,2,2,4,2,0,0,...,4,3,3,4,1,2,0,1,1,2
n1ed8fc36eda9df0,98,0.75,4,4,0,1,1,2,2,2,...,2,2,0,0,3,3,2,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nad4b893e7910343,108,0.50,3,0,2,1,1,2,1,3,...,0,3,2,2,2,1,4,3,2,3
n4c483117a13d704,151,0.50,1,3,0,2,2,2,1,4,...,0,3,1,2,4,3,2,4,3,1
nba238dc06d80658,105,0.50,1,3,0,0,1,2,2,2,...,2,0,1,3,3,0,2,3,1,3


Encoding the Numeric Values in the Target into corresponding labels (class 0 to class 4)

In [30]:
label_encoder = LabelEncoder()
label_encoder.fit(dataset['target'])
train_df['target'] = label_encoder.transform(train_df['target'])
val_df['target'] = label_encoder.transform(val_df['target'])

Label Map/Dictionary to store the mapping

In [31]:
labels_dict = {i: label_encoder.transform([i])[0] for i in label_encoder.classes_}
labels_dict

{0.0: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.0: 4}

In [32]:
train_df_x = train_df.drop(['target'], axis=1, inplace=False)
train_df_y = train_df['target']

In [33]:
train_df_x

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n02ccde8415c38aa,538,1,2,0,2,1,2,0,3,4,...,4,4,1,4,0,1,0,4,3,4
n2304b3e75684be7,538,4,3,4,0,0,4,4,0,4,...,2,2,4,3,0,3,1,0,2,0
n8f9f03f87006ca6,301,3,0,0,0,2,1,4,4,3,...,0,2,3,1,4,1,0,1,1,1
n329f28c70b122ca,516,1,4,1,1,0,2,3,0,3,...,4,1,4,3,4,1,2,2,4,0
neb7b6c30352e104,336,2,1,0,2,4,3,4,4,0,...,2,4,3,4,4,3,2,4,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
n26ad4563c77aadd,40,3,2,1,2,3,2,3,2,0,...,2,2,4,4,1,4,2,1,2,3
nd17176101e0c0d4,369,3,0,1,3,3,2,1,2,1,...,2,3,3,3,3,1,0,2,0,4
n68766810d412555,500,4,3,3,2,2,4,2,4,0,...,0,2,3,0,0,2,1,2,2,1


In [34]:
train_df_y

id
n02ccde8415c38aa    2
n2304b3e75684be7    3
n8f9f03f87006ca6    1
n329f28c70b122ca    2
neb7b6c30352e104    2
                   ..
n26ad4563c77aadd    2
nd17176101e0c0d4    2
n68766810d412555    2
na53b191b95a22b5    3
n7d36f87401b3463    2
Name: target, Length: 2197016, dtype: int64

In [35]:
val_df_x = val_df.drop(['target'], axis=1, inplace=False)
val_df_y = val_df['target']

In [36]:
val_df_x

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
nf313a6a2ba5d8ea,308,1,0,4,0,0,2,3,4,3,...,1,0,0,4,0,0,0,2,0,0
ne898d680d03f464,331,3,0,1,1,2,0,3,1,1,...,1,4,2,0,2,2,1,1,2,3
n986fb66ab539cb7,237,1,3,0,2,3,1,3,2,4,...,1,3,2,4,2,2,0,4,2,3
n3d14ff5f4be25e3,125,3,3,2,2,4,2,0,0,0,...,4,3,3,4,1,2,0,1,1,2
n1ed8fc36eda9df0,98,4,4,0,1,1,2,2,2,2,...,2,2,0,0,3,3,2,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nad4b893e7910343,108,3,0,2,1,1,2,1,3,3,...,0,3,2,2,2,1,4,3,2,3
n4c483117a13d704,151,1,3,0,2,2,2,1,4,4,...,0,3,1,2,4,3,2,4,3,1
nba238dc06d80658,105,1,3,0,0,1,2,2,2,0,...,2,0,1,3,3,0,2,3,1,3


In [37]:
val_df_y

id
nf313a6a2ba5d8ea    2
ne898d680d03f464    2
n986fb66ab539cb7    2
n3d14ff5f4be25e3    2
n1ed8fc36eda9df0    3
                   ..
nad4b893e7910343    2
n4c483117a13d704    2
nba238dc06d80658    2
n5e84b8fff8e7781    3
n37cfbcb6f5ef398    3
Name: target, Length: 549254, dtype: int64

In [38]:
del train_df
del val_df

gc.collect()

0

Undersampling the training & validation fractions to ensure a more balanced dataset, since NumerAi is imbalanced/skewed towards a value of 0.5 (class 2)

In [40]:
undersample = RandomUnderSampler(random_state=42)

In [41]:
train_df_x_resampled, train_df_y_resampled = undersample.fit_resample(train_df_x, train_df_y)

In [42]:
val_df_x_resampled, val_df_y_resampled = undersample.fit_resample(val_df_x, val_df_y)

In [43]:
del train_df_x
del val_df_x
del train_df_y
del val_df_y

gc.collect()

0

Converting data into floating type format

In [44]:
train_df_x_resampled = train_df_x_resampled.astype(np.float32)
val_df_x_resampled = val_df_x_resampled.astype(np.float32)

train_df_y_resampled = train_df_y_resampled.astype(np.float32)
val_df_y_resampled = val_df_y_resampled.astype(np.float32)

In [45]:
train_df_x_resampled

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
na07549d9cb0c978,247.0,2.0,4.0,0.0,3.0,1.0,1.0,0.0,0.0,2.0,...,3.0,3.0,4.0,1.0,3.0,2.0,0.0,3.0,3.0,0.0
neba7f32ece3086e,365.0,0.0,3.0,2.0,4.0,3.0,2.0,0.0,2.0,1.0,...,2.0,1.0,3.0,3.0,4.0,1.0,2.0,4.0,0.0,0.0
na7293ccee5af948,318.0,0.0,3.0,2.0,4.0,2.0,2.0,4.0,4.0,3.0,...,2.0,4.0,3.0,1.0,4.0,2.0,0.0,4.0,1.0,1.0
nf38c63dadffa0a3,233.0,0.0,3.0,3.0,4.0,0.0,2.0,4.0,4.0,1.0,...,2.0,4.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0
ne31c4087a4bf180,435.0,0.0,3.0,1.0,2.0,4.0,0.0,1.0,3.0,0.0,...,1.0,2.0,3.0,2.0,1.0,3.0,3.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
n4b1f1c5a85f36d3,267.0,2.0,1.0,1.0,3.0,1.0,1.0,3.0,4.0,2.0,...,4.0,1.0,1.0,3.0,1.0,3.0,0.0,4.0,3.0,3.0
n54d459f5fd5dde5,250.0,4.0,1.0,1.0,0.0,4.0,0.0,2.0,3.0,1.0,...,3.0,2.0,4.0,2.0,3.0,3.0,1.0,3.0,2.0,0.0
n6cf14468d0a8c44,526.0,0.0,4.0,2.0,0.0,0.0,4.0,1.0,1.0,3.0,...,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,4.0


In [46]:
val_df_x_resampled

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
ncdf26b8ce478ecf,574.0,1.0,0.0,4.0,0.0,0.0,3.0,2.0,3.0,0.0,...,1.0,1.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0
n4b4bf82266cda26,560.0,3.0,4.0,3.0,0.0,3.0,2.0,2.0,4.0,4.0,...,0.0,4.0,4.0,0.0,4.0,4.0,2.0,2.0,4.0,0.0
n61d5d7b00dc995f,379.0,2.0,0.0,4.0,1.0,0.0,3.0,0.0,3.0,3.0,...,2.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
n3162d17321ad479,569.0,0.0,4.0,4.0,0.0,1.0,0.0,3.0,4.0,4.0,...,2.0,2.0,2.0,1.0,4.0,2.0,2.0,0.0,4.0,0.0
n10d10d843b6046c,292.0,0.0,4.0,2.0,0.0,1.0,3.0,0.0,2.0,1.0,...,2.0,2.0,3.0,1.0,1.0,4.0,3.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
n051dbcb0462e3b6,114.0,2.0,2.0,2.0,0.0,0.0,2.0,4.0,2.0,2.0,...,1.0,3.0,4.0,3.0,4.0,4.0,2.0,3.0,4.0,2.0
n40f6c9a87c5afcc,5.0,4.0,2.0,4.0,2.0,1.0,2.0,3.0,2.0,0.0,...,2.0,3.0,2.0,4.0,0.0,4.0,2.0,3.0,0.0,0.0
need05ca67530d37,447.0,0.0,1.0,4.0,3.0,3.0,2.0,1.0,3.0,4.0,...,2.0,0.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,4.0


In [47]:
train_df_y_resampled

id
na07549d9cb0c978    0.0
neba7f32ece3086e    0.0
na7293ccee5af948    0.0
nf38c63dadffa0a3    0.0
ne31c4087a4bf180    0.0
                   ... 
n4b1f1c5a85f36d3    4.0
n54d459f5fd5dde5    4.0
n6cf14468d0a8c44    4.0
nb572d7239a224e5    4.0
n04135d14c62dced    4.0
Name: target, Length: 542805, dtype: float32

In [48]:
val_df_y_resampled

id
ncdf26b8ce478ecf    0.0
n4b4bf82266cda26    0.0
n61d5d7b00dc995f    0.0
n3162d17321ad479    0.0
n10d10d843b6046c    0.0
                   ... 
n051dbcb0462e3b6    4.0
n40f6c9a87c5afcc    4.0
need05ca67530d37    4.0
nba162f3f2e3cfba    4.0
nbd2485dc01a4c1a    4.0
Name: target, Length: 135535, dtype: float32

Function to extract the highly correlated features with respect to the predictions

In [49]:
def get_highly_correlated_features(x: pd.DataFrame, y, correlation_threshold: float = 0.03) -> list:
    if not isinstance(y, pd.Series):
        y = pd.Series(y, index=x.index)

    correlations = x.apply(lambda feature: feature.corr(y))
    highly_correlated_features = correlations[correlations.abs() > correlation_threshold].index.tolist()
    
    return highly_correlated_features

Function to neutralise the effect of highly correlated features with respect to the predictions by subtracting a proportion from the predictions

In [50]:
def neutralize_predictions(predictions: np.ndarray, x: pd.DataFrame, features_to_neutralize: list, proportion: float = 1.0) -> np.ndarray:
	predictions_df = pd.DataFrame(predictions, columns=['prediction'], index=x.index)
	neutralized_preds = predictions_df['prediction'].copy()
	
	for feature in features_to_neutralize:
		neutralizer = x[feature]
		adjustment = proportion * (neutralizer.dot(predictions_df['prediction']) / neutralizer.dot(neutralizer)) * neutralizer
		neutralized_preds -= adjustment
	# neutralizer = x
	# print(neutralizer.shape, neutralized_preds.shape)
	# adjustment = proportion * (neutralizer.dot(predictions_df['prediction']) / neutralizer.dot(neutralizer)) * neutralizer
	# neutralized_preds -= adjustment
   
	return neutralized_preds.values

Calculating the class weights to ensure a more balanced training for some of the weak classifiers/experts

In [53]:
classes = np.unique(train_df_y_resampled)
class_weights = compute_class_weight('balanced', classes=classes, y=train_df_y_resampled)
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}
sample_weights = np.array([class_weight_dict[label] for label in train_df_y_resampled])

Evaluation Functions

Function to compute Label Frequencies

In [ ]:
def label_frequency(predictions):
    unique, counts = np.unique(predictions, return_counts=True)
    label_frequencies = dict(zip(unique, counts))
    print("Label frequencies:", label_frequencies)

Function to compute NumerAi Correlation

In [ ]:
def numerai_corr(preds, target):
  ranked_preds = (preds.rank(method="average").values - 0.5) / preds.count()
  gauss_ranked_preds = stats.norm.ppf(ranked_preds)

  centered_target = target - target.mean()

  preds_p15 = np.sign(gauss_ranked_preds) * np.abs(gauss_ranked_preds) ** 1.5
  target_p15 = np.sign(centered_target) * np.abs(centered_target) ** 1.5

  return np.corrcoef(preds_p15, target_p15)[0, 1]

EXPERT-1 (XGBOOST CLASSIFIER)

In [54]:
'''
Commented out if the model is already trained over this & saved
'''

expert1 = get_xgboost()

expert1.fit(train_df_x_resampled, train_df_y_resampled, sample_weight=sample_weights)

Loading the Model

In [55]:
# with open('./saved_models/numerai_expert1.pkl', 'rb') as f:
#     expert1 = pickle.load(f)
# print("Model loaded successfully!")

Model loaded successfully!


Saving the Model

In [56]:
with open('./saved_models/numerai_expert1.pkl', 'wb') as f:
    pickle.dump(expert1, f)
print("Expert1 saved successfully!")


Expert1 saved successfully!


Generating Predictions from Expert-1

In [57]:
expert1_pred = expert1.predict(val_df_x_resampled)

In [58]:
expert1_pred

array([0, 0, 0, ..., 4, 4, 1])

Computing Label Frequencies in Prediction, Accuracy, Highly Correlated Features & Neutralising the Predictions

In [59]:
label_frequency(expert1_pred)

Label frequencies: {0: 31003, 1: 19210, 2: 34192, 3: 20200, 4: 30930}


In [60]:
acc = accuracy_score(expert1_pred, val_df_y_resampled)
print("Accuracy on Training Set: ", acc)

Accuracy on Training Set:  0.4346847677721622


In [61]:
highly_correlated_features = get_highly_correlated_features(val_df_x_resampled, expert1_pred, correlation_threshold=0.09)
print("Highly correlated features:", highly_correlated_features)

Highly correlated features: []


In [62]:
# expert1_pred = neutralize_predictions(predictions=expert1_pred, x=val_df_x_resampled, features_to_neutralize=highly_correlated_features, proportion=0.01)
# print("Neutralized predictions, Expert1:", expert1_pred)

Neutralized predictions, Expert1: [0 0 0 ... 4 4 1]


In [ ]:
expert1_pred = neutralize_predictions(predictions=expert1_pred, x=val_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert1:", expert1_pred)

EXPERT-2 (RANDOM FOREST CLASSIFIER)

In [63]:
'''
Commented out if the model is already trained over this & saved
'''

expert2 = get_random_forest()

expert2.fit(train_df_x_resampled, train_df_y_resampled, sample_weight=sample_weights)

Loading the Model

In [64]:
# with open('./saved_models/numerai_expert2.pkl', 'rb') as f:
#     expert2 = pickle.load(f)
# print("Model loaded successfully!")

Model loaded successfully!


Saving the Model

In [65]:
with open('./saved_models/numerai_expert2.pkl', 'wb') as f:
    pickle.dump(expert2, f)
print("Expert2 saved successfully!")

Expert2 saved successfully!


Generating Predictions from Expert-2

In [66]:
expert2_pred = expert2.predict(val_df_x_resampled)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    5.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    9.7s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   12.1s finished


In [67]:
expert2_pred

array([0., 3., 0., ..., 4., 3., 1.], dtype=float32)

Computing Label Frequencies in Prediction, Accuracy, Highly Correlated Features & Neutralising the Predictions

In [68]:
label_frequency(expert2_pred)

Label frequencies: {0.0: 30049, 1.0: 12036, 2.0: 47796, 3.0: 12200, 4.0: 33454}


In [69]:
acc = accuracy_score(expert2_pred, val_df_y_resampled)
print("Accuracy on Training Set: ", acc)

Accuracy on Training Set:  0.3458442468735013


In [70]:
highly_correlated_features = get_highly_correlated_features(val_df_x_resampled, expert2_pred, correlation_threshold=0.09)
print("Highly correlated features:", highly_correlated_features)

Highly correlated features: ['feature 42', 'feature 56', 'feature 61', 'feature 64', 'feature 65', 'feature 69', 'feature 74', 'feature 76', 'feature 81', 'feature 88', 'feature 91', 'feature 101', 'feature 113', 'feature 126', 'feature 127', 'feature 129', 'feature 133', 'feature 142', 'feature 157', 'feature 160', 'feature 168', 'feature 172', 'feature 184', 'feature 200', 'feature 208', 'feature 222', 'feature 232', 'feature 246', 'feature 263', 'feature 268', 'feature 276', 'feature 296', 'feature 302', 'feature 306', 'feature 329', 'feature 335', 'feature 346', 'feature 350', 'feature 352', 'feature 374', 'feature 406', 'feature 423', 'feature 424', 'feature 428', 'feature 431', 'feature 437', 'feature 451', 'feature 455', 'feature 459', 'feature 466', 'feature 475', 'feature 484', 'feature 501', 'feature 514', 'feature 537', 'feature 545', 'feature 546', 'feature 547', 'feature 552', 'feature 566', 'feature 573', 'feature 582', 'feature 588', 'feature 593', 'feature 615', 'featur

In [71]:
# expert2_pred = neutralize_predictions(predictions=expert2_pred, x=val_df_x_resampled, features_to_neutralize=highly_correlated_features, proportion=0.01)
# print("Neutralized predictions, Expert2:", expert2_pred)

Neutralized predictions, Expert2: [-0.7157442   1.4416975  -0.43838832 ...  3.3149312   2.3832166
 -0.6667897 ]


In [ ]:
expert2_pred = neutralize_predictions(predictions=expert2_pred, x=val_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert2:", expert2_pred)

EXPERT-3 (ADABOOST CLASSIFIER with DECISION TREE CLASSIFIER as BASE ESTIMATOR)

In [72]:
'''
Commented out if the model is already trained over this & saved
'''

expert3 = get_adaboost()

expert3.fit(train_df_x_resampled, train_df_y_resampled)

Loading the Model

In [73]:
# with open('./saved_models/numerai_expert3.pkl', 'rb') as f:
#     expert3 = pickle.load(f)
# print("Model loaded successfully!")

Model loaded successfully!


Saving the Model

In [74]:
with open('./saved_models/numerai_expert3.pkl', 'wb') as f:
    pickle.dump(expert3, f)
print("Expert3 saved successfully!")

Expert3 saved successfully!


Generating Predictions from Expert-3

In [75]:
expert3_pred = expert3.predict(val_df_x_resampled)

In [76]:
expert3_pred

array([0., 3., 4., ..., 4., 1., 1.], dtype=float32)

Computing Label Frequencies in Prediction, Accuracy, Highly Correlated Features & Neutralising the Predictions

In [77]:
label_frequency(expert3_pred)

Label frequencies: {0.0: 27770, 1.0: 13593, 2.0: 44088, 3.0: 16264, 4.0: 33820}


In [78]:
acc = accuracy_score(expert3_pred, val_df_y_resampled)
print("Accuracy on Training Set: ", acc)

Accuracy on Training Set:  0.29076622274689196


In [79]:
highly_correlated_features = get_highly_correlated_features(val_df_x_resampled, expert3_pred, correlation_threshold=0.09)
print("Highly correlated features:", highly_correlated_features)

Highly correlated features: ['feature 4', 'feature 28', 'feature 36', 'feature 39', 'feature 49', 'feature 57', 'feature 69', 'feature 72', 'feature 124', 'feature 131', 'feature 133', 'feature 145', 'feature 146', 'feature 151', 'feature 157', 'feature 158', 'feature 159', 'feature 168', 'feature 175', 'feature 192', 'feature 219', 'feature 221', 'feature 244', 'feature 245', 'feature 254', 'feature 257', 'feature 268', 'feature 279', 'feature 299', 'feature 333', 'feature 339', 'feature 342', 'feature 344', 'feature 351', 'feature 382', 'feature 397', 'feature 402', 'feature 422', 'feature 427', 'feature 437', 'feature 442', 'feature 444', 'feature 458', 'feature 470', 'feature 477', 'feature 501', 'feature 509', 'feature 512', 'feature 533', 'feature 535', 'feature 543', 'feature 546', 'feature 547', 'feature 551', 'feature 557', 'feature 558', 'feature 582', 'feature 587', 'feature 589', 'feature 596', 'feature 618', 'feature 632', 'feature 639', 'feature 649', 'feature 656', 'feat

In [80]:
# expert3_pred = neutralize_predictions(predictions=expert3_pred, x=val_df_x_resampled, features_to_neutralize=highly_correlated_features, proportion=0.01)
# print("Neutralized predictions, Expert3:", expert3_pred)

Neutralized predictions, Expert3: [-0.7431115   1.756854    3.0819592  ...  2.7908666  -0.15753624
  0.10444321]


In [ ]:
expert3_pred = neutralize_predictions(predictions=expert3_pred, x=val_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert3:", expert3_pred)

EXPERT-4 (LOGISTIC REGRESSION)

In [81]:
'''
Commented out if the model is already trained over this & saved
'''

expert4 = get_logistic_regression()

expert4.fit(train_df_x_resampled, train_df_y_resampled)

Loading the Model

In [82]:
# with open('./saved_models/numerai_expert4.pkl', 'rb') as f:
#     expert4 = pickle.load(f)
# print("Model loaded successfully!")

Model loaded successfully!


Saving the Model

In [83]:
with open('./saved_models/numerai_expert4.pkl', 'wb') as f:
    pickle.dump(expert4, f)
print("Expert4 saved successfully!")

Expert4 saved successfully!


Generating Predictions from Expert-4

In [84]:
expert4_pred = expert4.predict(val_df_x_resampled)

In [85]:
expert4_pred

array([4., 1., 0., ..., 0., 3., 1.], dtype=float32)

Computing Label Frequencies in Prediction, Accuracy, Highly Correlated Features & Neutralising the Predictions

In [86]:
label_frequency(expert4_pred)

Label frequencies: {0.0: 31387, 1.0: 15920, 2.0: 41091, 3.0: 16596, 4.0: 30541}


In [87]:
acc = accuracy_score(expert4_pred, val_df_y_resampled)
print("Accuracy on Training Set: ", acc)

Accuracy on Training Set:  0.29539233408344706


In [88]:
highly_correlated_features = get_highly_correlated_features(val_df_x_resampled, expert4_pred, correlation_threshold=0.09)
print("Highly correlated features:", highly_correlated_features)

Highly correlated features: ['feature 17', 'feature 45', 'feature 72', 'feature 133', 'feature 168', 'feature 254', 'feature 286', 'feature 287', 'feature 288', 'feature 363', 'feature 372', 'feature 379', 'feature 389', 'feature 431', 'feature 517', 'feature 546', 'feature 618', 'feature 640', 'feature 657', 'feature 666', 'feature 671', 'feature 676', 'feature 698']


In [89]:
# expert4_pred = neutralize_predictions(predictions=expert4_pred, x=val_df_x_resampled, features_to_neutralize=highly_correlated_features, proportion=0.01)
# print("Neutralized predictions, Expert4:", expert4_pred)

Neutralized predictions, Expert4: [ 3.742752    0.6304167  -0.18413977 ... -0.35999796  2.648405
  0.6216319 ]


In [ ]:
expert4_pred = neutralize_predictions(predictions=expert4_pred, x=val_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert4:", expert4_pred)

EXPERT-5 (CATBOOST CLASSIFIER)

In [91]:
'''
Commented out if the model is already trained over this & saved
'''

expert5 = get_catboost()

expert5.fit(train_df_x_resampled, train_df_y_resampled)

Loading the Model

In [92]:
# with open('./saved_models/numerai_expert5.pkl', 'rb') as f:
#     expert5 = pickle.load(f)
# print("Model loaded successfully!")

Model loaded successfully!


Saving the Model

In [93]:
with open('./saved_models/numerai_expert5.pkl', 'wb') as f:
    pickle.dump(expert5, f)
print("Expert5 saved successfully!")

Expert5 saved successfully!


Generating Predictions from Expert-5

In [94]:
expert5_pred = expert5.predict(val_df_x_resampled)

In [95]:
if expert5_pred.ndim > 1:
    expert5_pred = expert5_pred.ravel()

In [96]:
expert5_pred

array([0., 3., 0., ..., 4., 3., 1.])

Computing Label Frequencies in Prediction, Accuracy, Highly Correlated Features & Neutralising the Predictions

In [97]:
label_frequency(expert5_pred)

Label frequencies: {0.0: 30896, 1.0: 16431, 2.0: 38741, 3.0: 17887, 4.0: 31580}


In [98]:
acc = accuracy_score(expert5_pred, val_df_y_resampled)
print("Accuracy on Training Set: ", acc)

Accuracy on Training Set:  0.3158372376139005


In [99]:
highly_correlated_features = get_highly_correlated_features(val_df_x_resampled, expert5_pred, correlation_threshold=0.09)
print("Highly correlated features:", highly_correlated_features)

Highly correlated features: ['feature 17', 'feature 45', 'feature 69', 'feature 72', 'feature 133', 'feature 168', 'feature 254', 'feature 268', 'feature 286', 'feature 287', 'feature 288', 'feature 363', 'feature 437', 'feature 477', 'feature 517', 'feature 546', 'feature 640', 'feature 657', 'feature 671', 'feature 676']


In [100]:
# expert5_pred = neutralize_predictions(predictions=expert5_pred, x=val_df_x_resampled, features_to_neutralize=highly_correlated_features, proportion=0.01)
# print("Neutralized predictions, Expert5:", expert5_pred)

Neutralized predictions, Expert5: [-0.1755911   2.67527935 -0.25528764 ...  3.73431531  2.75650493
  0.63843689]


In [ ]:
expert5_pred = neutralize_predictions(predictions=expert5_pred, x=val_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert5:", expert5_pred)

EXPERT-6 (HISTOGRAM-BASED GRADIENT BOOST CLASSIFIER)

In [ ]:
'''
Commented out if the model is already trained over this & saved
'''

expert6 = get_histogram_gb()

expert6.fit(train_df_x_resampled, train_df_y_resampled, sample_weight=sample_weights)

Loading the Model

In [102]:
# with open('./saved_models/numerai_expert6.pkl', 'rb') as f:
#     expert6 = pickle.load(f)
# print("Model loaded successfully!")

Saving the Model

In [103]:
with open('./saved_models/numerai_expert6.pkl', 'wb') as f:
    pickle.dump(expert6, f)
print("Expert6 saved successfully!")

Expert6 saved successfully!


Generating Predictions from Expert-6

In [104]:
expert6_pred = expert6.predict(val_df_x_resampled)

In [105]:
expert6_pred

array([0., 1., 4., ..., 3., 2., 0.], dtype=float32)

Computing Label Frequencies in Prediction, Accuracy, Highly Correlated Features & Neutralising the Predictions

In [106]:
label_frequency(expert6_pred)

Label frequencies: {0.0: 29463, 1.0: 14034, 2.0: 43783, 3.0: 15921, 4.0: 32334}


In [107]:
acc = accuracy_score(expert6_pred, val_df_y_resampled)
print("Accuracy on Training Set: ", acc)

Accuracy on Training Set:  0.28636883461836427


In [108]:
highly_correlated_features = get_highly_correlated_features(val_df_x_resampled, expert6_pred, correlation_threshold=0.09)
print("Highly correlated features:", highly_correlated_features)

Highly correlated features: []


In [109]:
# expert6_pred = neutralize_predictions(predictions=expert6_pred, x=val_df_x_resampled, features_to_neutralize=highly_correlated_features, proportion=0.01)
# print("Neutralized predictions, Expert6:", expert6_pred)

Neutralized predictions, Expert6: [0. 1. 4. ... 3. 2. 0.]


In [ ]:
expert6_pred = neutralize_predictions(predictions=expert6_pred, x=val_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert6:", expert6_pred)

META-MODEL (LIGHTGBM)

In [110]:
meta_model = get_lightgbm()

In [ ]:
meta_val_x = np.column_stack((expert1_pred, expert2_pred, expert3_pred, expert4_pred, expert5_pred, expert6_pred))

In [111]:
meta_val_x

array([[ 0.        , -0.7157442 , -0.74311149,  3.74275208, -0.1755911 ,
         0.        ],
       [ 0.        ,  1.44169748,  1.75685406,  0.63041669,  2.67527935,
         1.        ],
       [ 0.        , -0.43838832,  3.08195925, -0.18413977, -0.25528764,
         4.        ],
       ...,
       [ 4.        ,  3.31493115,  2.79086661, -0.35999796,  3.73431531,
         3.        ],
       [ 4.        ,  2.38321662, -0.15753624,  2.64840508,  2.75650493,
         2.        ],
       [ 1.        , -0.66678971,  0.10444321,  0.62163192,  0.63843689,
         0.        ]])

In [112]:
meta_model.fit(meta_val_x, val_df_y_resampled)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005744 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1030
[LightGBM] [Info] Number of data points in the train set: 135535, number of used features: 6
[LightGBM] [Info] Start training from score 2.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

LGBMRegressor(colsample_bytree=0.1, learning_rate=0.01, max_depth=10,
              n_estimators=10000, num_leaves=1024, objective='regression',
              random_state=42)

Loading the Meta-Model

In [ ]:
# with open('./saved_models/numerai_meta_model.pkl', 'rb') as f:
#     meta_model = pickle.load(f)
# print("Meta Model loaded successfully!")

Saving the Meta-Model

In [113]:
with open('./saved_models/numerai_meta_model.pkl', 'wb') as f:
    pickle.dump(meta_model, f)
print("Meta Model saved successfully!")

Meta Model saved successfully!


Testing Performance on Trained Data

In [114]:
meta_y_pred = meta_model.predict(meta_val_x)

In [115]:
meta_y_pred

array([0.97908834, 1.04324704, 1.06969933, ..., 2.92813462, 3.03981607,
       1.54116925])

In [116]:
bins = [0.5, 1.5, 2.5, 3.5]

rounded_predictions = np.digitize(meta_y_pred, bins)

In [117]:
rounded_predictions

array([1, 1, 1, ..., 3, 3, 2])

Computing Relevant Evaluation Metrics

In [118]:
label_frequency(rounded_predictions)

Label frequencies: {0: 31, 1: 36803, 2: 62113, 3: 36569, 4: 19}


In [119]:
acc = accuracy_score(rounded_predictions, val_df_y_resampled)
print("Accuracy on Training Set: ", acc)

Accuracy on Training Set:  0.2581178293429741


Pearson's Correlation

In [120]:
pearson_corr, _ = stats.pearsonr(rounded_predictions, val_df_y_resampled)
print("Pearson Correlation:", pearson_corr)

Pearson Correlation: 0.4540510949801483


Computing the NumerAi's Correlation Metric

In [ ]:
rounded_predictions = pd.Series(rounded_predictions)

In [122]:
actual_corr = numerai_corr(rounded_predictions, val_df_y_resampled)
actual_corr

0.45786088424761245